In [1]:
import tensorflow as tf
import numpy as np

In [8]:
a=np.ones((2,3))
print(a)
np.max(a)

[[ 1.  1.  1.]
 [ 1.  1.  1.]]


1.0

In [2]:
with tf.variable_scope('foo') as foo:
    a=tf.constant(0, name='a')
    biases = tf.get_variable(shape=[512],initializer=tf.constant_initializer(0.0),  name='biases')
    print(a.name,)
    print(biases.name)
with tf.variable_scope(foo, reuse=True):
    c=tf.constant('2', name='c')
    print(c.name)

foo/a:0
foo/biases:0
foo_1/c:0


In [21]:
a=tf.constant(10)
b=tf.constant(2)
c=tf.div(a, b)
#init_op=tf.global_variables_initializer()

In [22]:
with tf.Session() as sess:
    print a.eval()
    sess.run(init_op)
    print(sess.run(c))

10
5


## network variable_name test

In [12]:
x=tf.constant(1)
w=tf.constant(2)
o=tf.multiply(x,w)
o=tf.multiply(o,w)
o=tf.multiply(o,w)
o=tf.multiply(o,w)
print o,w
with tf.Session() as sess:
    tf.global_variables_initializer()
    r=sess.run(o)
    print r

Tensor("Mul_15:0", shape=(), dtype=int32) Tensor("Const_9:0", shape=(), dtype=int32)
16


## multi returns

In [5]:
a=tf.constant(1)
b=tf.constant(2)
o1=tf.multiply(a,b)
o2=tf.multiply(o1,b)
o3=tf.multiply(o2,b)
with tf.Session() as sess:
    # how many times does o1=tf.muliply run?
    print(sess.run([o1,o2,o3]))

[2, 4, 8]


In [10]:
hello=tf.constant('hello')
with tf.Graph().as_default():
    hi=tf.constant("hi")
    print(hi.graph)
    print(hello.graph)

# Saver

## 1-simple example

In [2]:
# in this method, when resotre variables, the coresponding Variable should be declare first.
# save
with tf.variable_scope('train'):
    w=tf.Variable([[1,1,1],[2,2,2]],dtype = tf.float32,name='w')
    b = tf.Variable([[0,1,2]],dtype = tf.float32,name='b')

init=tf.global_variables_initializer()
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run([w,b]))
    print(w.name, b.name)
    save_path=saver.save(sess, 'save/model01.ckpt')

[array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.]], dtype=float32), array([[ 0.,  1.,  2.]], dtype=float32)]
train/w:0 train/b:0


In [2]:
# load! MUST firstly restart/refresh the kernel. Otherwise will got NAME INTERRUPTION.
# declare variables with identical SHAPE,DTYPE,NAME. 
with tf.variable_scope('train'):
    w=tf.Variable(tf.truncated_normal(shape=(2,3)), dtype=tf.float32, name='w')
    b=tf.Variable(tf.truncated_normal(shape=(1,3)),dtype = tf.float32,name='b')
    
saver=tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "save/model01.ckpt")
    print(sess.run(w))

INFO:tensorflow:Restoring parameters from save/model.ckpt
[[ 1.  1.  1.]
 [ 2.  2.  2.]]


## 2-another example   -restore graph

In [2]:
# save. 
x1=tf.placeholder(tf.float32, [], name='x1')
x2=tf.placeholder(tf.float32, [], name='x2')
b=tf.Variable(2.0, name='bias')

feed_dict={x1:4, x2:8}
y1=tf.add(x1, x2)
y2=tf.multiply(x1, x2)
y3=tf.add(y2, b, name='output') # when restore, this name will be used. so define a name.
# or 
# tf.add_to_collections('outputs',y3)
# when restore, y3=tf.get_collection('outputs')[0]

saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run([y3, y2, y1], feed_dict))
    saver.save(sess, 'save/model02.ckpt')

[34.0, 32.0, 12.0]


In [2]:
# restore. Restart the kernel first.
with tf.Session() as sess:
    saver=tf.train.import_meta_graph('./save/model02.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./save'))
    
    # test variable
    print(sess.run('bias:0'))
    # test graph
    graph=tf.get_default_graph()
    x1=graph.get_tensor_by_name("x1:0")
    x2=graph.get_tensor_by_name("x2:0")
    feed_dict={x1:3, x2:4}
    
    out=graph.get_tensor_by_name("output:0")
    # or out=graph.get_operation_by_name('output').outputs[0]
    
    print(sess.run(out, feed_dict=feed_dict))

INFO:tensorflow:Restoring parameters from ./save/model02.ckpt
2.0
14.0


In [40]:
class net:
    def __init__(self):
        self.input=tf.placeholder(tf.float32,[])
        #self.forward(self.input)
    def forward(self, img):
        w=tf.Variable(1.0, name='w')
        o1=tf.add(img, w)
        o2=tf.add(o1, w)
        print('ch01')
        o3=tf.add(o2, w)
        print('ch02')
        return [o1,o2,o3]

x=tf.placeholder(tf.float32,[])
m=net()
out=m.forward(x)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(out,feed_dict={x:0.0}))

ch01
ch02
[1.0, 2.0, 3.0]


In [3]:
x=[[2,3],[4,5]]
np.array(x)

array([[2, 3],
       [4, 5]])